In [11]:
import numpy as np

class Sigmoid:
    def __init__(self):
        self.params = []
        self.grads = []
        self.out = None
        
    def forward(self,x):
        out = 1/(1+np.exp(-1))
        self.out = out
        return out
    
    def backward(self,dout):
        dx = dout*(1.0-self.out)*self.out
        return dx

In [12]:
class Affine:
    def __init__(self,W,b):
        self.params = [W,b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
    
    def forward(self,x):
        W, b = self.params
        out = np.dot(x,W)+b
        self.x = x
        return out
    
    def backward(self,dout):
        W, b = self.params
        dx = np.dot(dout,W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)
        
        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

In [6]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 重みとバイアスの初期化
        W1 = np.random.randn(I,H)
        b1 = np.random.randn(H)
        W2 = np.random.randn(H,O)
        b2 = np.random.randn(O)
        
        # レイヤの生成
        self.layers = [
            Affine(W1,b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        
        # 全ての重みをリストにまとめる
        self.params = []
        for layer in self.layers:
            self.params += layer.params
        
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
            return x

In [8]:
x = np.random.randn(10,2)
model = TwoLayerNet(2,4,3)
s = model.predict(x)

In [9]:
print(s)

[[ 1.80594621  1.09722484  0.70259079 -3.12833094]
 [-1.66902612  0.31643616  0.19164814 -0.13929605]
 [ 0.42757856  7.09569264 -5.66773862 -1.43486839]
 [-1.80482323 -0.38128375  0.8240273  -0.07620283]
 [-0.05232176  1.13252473 -0.01338569 -1.49346605]
 [ 1.60842447 -1.32834962  3.00170188 -3.15013048]
 [-0.28040062  1.42083119 -0.37891122 -1.26994554]
 [ 1.08550241 -1.15270167  2.63820074 -2.67673373]
 [-0.86064352  3.44127009 -2.56713229 -0.5976903 ]
 [-0.29916606  1.39420667 -0.3597614  -1.25560822]]


In [13]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None

    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x
        return out

    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

In [15]:
class SoftmaxWithLoss:
    def __init__(self):
        self.params, self.grads = [], []
        self.y = None  # softmaxの出力
        self.t = None  # 教師ラベル

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)

        # 教師ラベルがone-hotベクトルの場合、正解のインデックスに変換
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)

        loss = cross_entropy_error(self.y, self.t)
        return loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]

        dx = self.y.copy()
        dx[np.arange(batch_size), self.t] -= 1
        dx *= dout
        dx = dx / batch_size

        return dx

In [16]:
def softmax(x):
    if x.ndim == 2:
        x = x - x.max(axis=1, keepdims=True)
        x = np.exp(x)
        x /= x.sum(axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x)
        x = np.exp(x) / np.sum(np.exp(x))

    return x


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]

    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


In [17]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr*grads[i]

In [ ]:
class TwoLayerNet:
    def __init__(self,input_size,hidden_size,output_size):
        I, H, O = input_size, hidden_size, output_size
        
        # 重みとバイアスの初期化
        W1 = 0.01*np.random.randn(I,H)
        b1 = np.zeros(H)
        W2 = 0.01*np.random.randn(H,O)
        b2 = np.zeros(O)
        
        # レイヤの生成
        self.layers = [
            Affine(W1,b1),
            Sigmoid(),
            Affine(W2, b2)
        ]
        self.loss_layer = SoftmaxWithLoss()
        
        # 全ての重みと勾配をリストにまとめる
        self.params = []
        self.grads = []
        for layer,grads in self.layers,self.grads:
            self.params += layer.params
            self.grads += layer.grads
        
    def predict(self, x):
        for layer in self.layers:
            x = layer.forward(x)
            return x
        
    def forward(self,x,t):
        score = self.predict(x)
        loss = self.loss_layer.forward(score,t)
        return loss
    
    def backward(self,dout=1):
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout